# GTFS schedule tables demo

* Take a look at 1 route for 1 operator, and see how `trips`, `shapes`, `stops`, and `stop_times` look

In [ ]:
import geopandas as gpd
import pandas as pd

date = "2023-08-16"
GCS_FILE_PATH = ("gs://calitp-analytics-data/data-analyses/"
                 "rt_delay/compiled_cached_views/")

In [ ]:
trips = pd.read_parquet(
    f"{GCS_FILE_PATH}trips_{date}.parquet"
)

trips = trips[(trips.name == "Sacramento Schedule") &
              (trips.route_id == "507")][
    ["feed_key", "name", "trip_id", 
     "route_id", "direction_id", 
     "shape_array_key", "shape_id"]
].reset_index(drop=True)

In [ ]:
sac_feed = trips.feed_key.iloc[0]
sac_feed

In [ ]:
sac_trips = trips.trip_id.unique().tolist()
len(sac_trips)

In [ ]:
sac_shapes = trips.shape_array_key.unique().tolist()
sac_shapes

In [ ]:
shapes = gpd.read_parquet(
    f"{GCS_FILE_PATH}routelines_{date}.parquet"
)

shapes = shapes[
    shapes.shape_array_key.isin(sac_shapes)][
    ["shape_array_key", "shape_id", "geometry"]
].reset_index(drop=True)

In [ ]:
stop_times = pd.read_parquet(
    f"{GCS_FILE_PATH}st_{date}.parquet"
)

stop_times = stop_times[(stop_times.feed_key == sac_feed) & 
                        (stop_times.trip_id.isin(sac_trips))
                       ][["feed_key", "trip_id", 
                          "stop_id", "stop_sequence", 
                          "arrival_sec"]]

In [ ]:
sac_stops = stop_times.stop_id.unique().tolist()
len(sac_stops)

In [ ]:
stops = gpd.read_parquet(
    f"{GCS_FILE_PATH}stops_{date}.parquet"
)

stops = stops[(stops.feed_key==sac_feed) & 
              (stops.stop_id.isin(sac_stops))
             ][
    ["feed_key", "stop_id", "geometry"]
].reset_index(drop=True)

In [ ]:
trips.shape, stops.shape, stop_times.shape, shapes.shape

In [ ]:
trips.head()

In [ ]:
trips.direction_id.value_counts()

In [ ]:
trips.shape_array_key.value_counts()

In [ ]:
shapes.head(1).explore(
    "shape_id",
    tiles = "CartoDB Positron",
    categorical = True
)

In [ ]:
shapes.tail(1).explore(
    "shape_id",
    tiles = "CartoDB Positron",
    categorical = True
)

In [ ]:
stops.head()

In [ ]:
stops.explore("stop_id", tiles = "CartoDB Positron")

In [ ]:
stop_times.sort_values(["trip_id", "stop_sequence"]).head(20)

In [ ]:
pd.merge(
    stop_times,
    stops,
    on = ["feed_key", "stop_id"]
)[["feed_key", "trip_id", "stop_id", "geometry"]].sort_values("trip_id")

In [ ]:
pd.merge(
    shapes,
    trips,
    on = ["shape_array_key", "shape_id"]
)[["feed_key", "shape_array_key", "shape_id", "trip_id", "geometry"]]